In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.constants as cst
from scipy.optimize import curve_fit
plt.rcParams.update({'font.size': 14})
from scipy import interpolate
from symfit import parameters, variables, Model, Fit, exp
from symfit.core.minimizers import DifferentialEvolution, LBFGSB
import scipy.constants as cst

# USE SYMPY 1.6.2 (and  probably new environment)

# From Pressure to Tension

In [ ]:
a = 5e-6 # Patch radius [m]
k = 0.24 # Area Compression modulus [N.m^-1]

In [ ]:
def displacement_solver(Ps, a, k):
    """
        Find the displacement for a given applied Pressure
        
        Parameters
        ----------
        
        Ps : float
            Pressure attributed to the circumferential tension 
            per unit length in the membrane [Pa]
        a : float
                Patch radius  [m]
        k : float
            Area Compression modulus [N.m^-1]
    
        Returns
        -------
        Z : array
            Patch displacement [m]
        """
    params = [2*k,-Ps*a**2,0,-Ps*a**4]

    Z = np.roots(params)
    Z = Z[np.isreal(Z)].real

    return Z[0]


In [ ]:
def set_R_curv(Z):
    R = (a**2 + Z**2) / (2*Z)
    return R

In [ ]:
def set_tension(R,Ps):
    Ts = (R * Ps)

    return Ts 

In [ ]:
Ps_mmHg = 15
Ps_PA = Ps_mmHg * 133.322 
Z_15mmHg = displacement_solver(Ps_PA,a,k)
R_curv = set_R_curv(Z_15mmHg)
Ts = set_tension(R_curv,Ps_PA)
print(Ts)

In [ ]:
import numpy as np

def coeff_of_determination(y, f):
    """
    This function computes the coefficient of determination.
        Input:
            y: observed data
            f: fitted/predicted value

        Output:
            R2: coefficient of determination
    """
    mean_y = np.mean(y)

    # Residual sum of squares
    SSres = np.sum((y-f)**2)

    # Total sum of squares
    SStot = np.sum((y-mean_y)**2)

    # Coefficient of determination
    R2 = 1 - (SSres/SStot)

    return R2

In [ ]:
def adjusted_coeff_determ(y, f, n, k):
    """
    This function computes the coefficient of determination.
        Input:
            y: observed data
            f: fitted/predicted value
            n: number of observations
            k: number of predictor variables

        Output:
            adjust_R2: ADJUSTED coefficient of determination
    """

    R2 = coeff_of_determination(y, f)
    adjust_R2 = 1- ((1-R2)*(n-1)/(n-k-1))

    return adjust_R2

# 1. Experimental Data

## 1.1. Load Data

In [ ]:
#from scipy.optimize import fmin as simplex
#DEFINING CONSTANTS
R =cst.R #gas constant [J mol^−1 K^−1]
F= 96485.33212 #[C mol^-1] Faraday constant
T = 273.15 + 25 # [kelvin] temperature const

# Compute reveral potential, using Nernst equation: 
ki = 150e-3 #[M] intracellular concentration
ko = 15e-3   #[M] extracellular concentration
z = 1
E_k = ((R*T)/(z*F))*np.log(ko/ki)*1e3
print(f'Reversal potential computed with Nerst equation:{E_k:3f} mV')

In [ ]:
V_hold = 0 # [mV]
P_arr = [2.5, 5, 7.5, 10, 12.5, 15, 17.5, 20, 22.5, 25]
P_arr_str = [str(elem) for elem in P_arr]
PATH_TO_DATA='Data/Mechanosensation/'

In [ ]:
I_data = {P_arr: pd.read_csv(PATH_TO_DATA + P_arr + '.csv', header=0, names=['x', 'y'], comment='#') 
for P_arr in P_arr_str}

In [ ]:
# Look at what the data I extracted look like
fig, ax= plt.subplots(1,1, figsize= (12,10))
for idx, value in enumerate(P_arr_str):
    plt.plot(I_data[value].x, I_data[value].y, label='P = ' + value + " mmHg")
plt.xlabel("t [ms]")
plt.ylabel("Current [nA]")
plt.grid()
plt.legend(loc='upper right')

## 1.2. Compute the conductance:


In [ ]:
#Fitted parameters from Voltage fitting
Am    =    2.756769e-02 
Vmm   =    7.704963e+01
b1m   =    -2.440201e+01 
b2m   =    4.802552e+01 
c1m   =    -1.712336e-01 
c2m    =   4.493341e-02 
d1m     =  4.181831e-03 
d2m     =  4.526902e-03 
g_bar  =    8.256491e-02 

In [ ]:
V_amp = np.arange(-100,120,20)
V_amp_str = [str(elem) for elem in V_amp]

In [ ]:
g_data = {P_arr: (I_data[P_arr].y/(V_hold-E_k)).to_frame().join(I_data[P_arr].x) for P_arr in P_arr_str}

In [ ]:
t_on, t_off = 10.1, 120
g_data_on = {P_arr: (I_data[P_arr][(I_data[P_arr].x>=t_on) & (I_data[P_arr].x <= t_off)].y/(V_hold-E_k)).to_frame().join(I_data[P_arr].x)
for P_arr in P_arr_str}

In [ ]:
# Look at what the computed conductances look like
fig, ax= plt.subplots(1,1, figsize= (12,10))

for idx, value in enumerate(P_arr_str):
    plt.plot(g_data[value].x, g_data[value].y, label='P = ' + value + " mmHg")
    
plt.xlabel("t [ms]")
plt.ylabel("Conductance [uS]")
plt.grid()
plt.legend(loc='upper right')

In [ ]:
# Look at what the computed conductances (pressure on) look like
fig, ax= plt.subplots(1,1, figsize= (12,10))

for idx, value in enumerate(P_arr_str):
    plt.plot(g_data_on[value].x, g_data_on[value].y, label='P = ' + value + " mmHg")
    
plt.xlabel("t [ms]")
plt.ylabel("Conductance [uS]")
plt.grid()
plt.legend(loc='upper right');

## 1.4. Fitting with fitted h0


In [ ]:
#Define the model with symfit
V_m0 = 0

x = variables(','.join((f'x{i}' for i in range(len(P_arr)))))
y = variables(','.join([f'y{i}' for i in range(len(P_arr))]))
gamma, delta_S, C, D, Ah, density, z1, z2 = parameters('gamma, delta_S, C, D, Ah, density, z1, z2')

density.min, density.max = 1, 150
gamma.min, gamma.max = 0, 1
delta_S.min, delta_S.max = 0, 200
C.min, C.max = -500,500
D.min, D.max = -500,500
Ah.min, Ah.max = 0.0001, 0.1
z1.min, z1.max = -10000, 10000
z2.min, z2.max = -10000, 10000 

alpham0 = (Am*exp(-(b1m*(V_m0-Vmm) + c1m*(V_m0-Vmm)**2 + d1m*(V_m0-Vmm)**3)/(R*T)))
betam0 = (Am*exp(-(b2m*(V_m0-Vmm) + c2m*(V_m0-Vmm)**2 + d2m*(V_m0-Vmm)**3)/(R*T)))
alphah0 = (exp(-(z1)/(R*T)))
betah0 = (exp(-(z2)/(R*T)))

m0 = alpham0/(alpham0+betam0) 
h0 = alphah0/(alphah0+betah0)

In [ ]:
Ps_Pa = [133.322*P for P in P_arr]  # mmHg to Pa
Z = [displacement_solver(Ps_Pa,a,k) for Ps_Pa in Ps_Pa]
R_curv = [set_R_curv(Z) for Z in Z]
Ts = [set_tension(R_curv, Ps_Pa) for R_curv, Ps_Pa in zip(R_curv,Ps_Pa)]

alpham = [Am*exp(-(b1m*(V_hold-Vmm) + c1m*(V_hold-Vmm)**2 + d1m*(V_hold-Vmm)**3 - gamma*cst.N_A*Ts*delta_S*10**(-20))/(R*T)) for Ts in Ts]
betam = [Am*exp(-(b2m*(V_hold-Vmm) + c2m*(V_hold-Vmm)**2 + d2m*(V_hold-Vmm)**3 - (gamma-1)*cst.N_A*Ts*delta_S*10**(-20))/(R*T)) for Ts in Ts]
alphah = [Ah*exp(-(z1 - C*cst.N_A*Ts*10**(-20))/(R*T)) for Ts in Ts]
betah = [Ah*exp(-(z2 - D*cst.N_A*Ts*10**(-20))/(R*T)) for Ts in Ts]

m_inf = [alpham/(alpham+betam) for alpham, betam in zip(alpham, betam)]
tau_m = [1/(alpham+betam) for alpham, betam in zip(alpham, betam)]
h_inf = [alphah/(alphah+betah) for alphah, betah in zip(alphah, betah)]
tau_h = [1/(alphah+betah) for alphah, betah in zip(alphah, betah)]

m = [m_inf - (m_inf - m0)*exp(-(x-t_on)/tau_m) for m_inf, x, tau_m in zip(m_inf,tau_m, x)]
h = [h_inf - (h_inf - h0)*exp(-(x-t_on)/tau_h) for h_inf, tau_h, x in zip(h_inf, tau_h, x)]

model = Model({y: density*g_bar*h*m for y, h, m in zip(y, h, m)})

In [ ]:
kwargs_x = {f'x{i}': g_data_on[P_arr].x.to_numpy() for i, P_arr in zip(range(len(P_arr_str)),P_arr_str)}
kwargs_y = {f'y{i}': g_data_on[P_arr].y.to_numpy() for i, P_arr in zip(range(len(P_arr_str)),P_arr_str)}

Runtype = 'full'

if Runtype == 'full':
    # Full run
    fit = Fit(model, **(kwargs_x | kwargs_y), minimizer= [DifferentialEvolution, LBFGSB])
    fit_result = fit.execute(DifferentialEvolution={'popsize': 50, 'recombination': 0.9, 'workers':-1})
elif Runtype == 'fast':
    # Fast 
    fit = Fit(model, **(kwargs_x | kwargs_y), minimizer= [LBFGSB])
    fit_result = fit.execute()

In [ ]:
print(fit_result)

In [ ]:
y_fit = model(**kwargs_x,**fit_result.params)

In [ ]:
#Plot the fitted curves
fig, ax= plt.subplots(len(P_arr), 1, constrained_layout=True, figsize=(15, 30))

for idx, value in enumerate(P_arr_str):
    
    ax[idx].plot(g_data_on[value].x, g_data_on[value].y, label= 'P = ' + value + " mmHg")
    ax[idx].plot(g_data_on[value].x, y_fit[idx], '--', label = 'fitted curve')
    ax[idx].set_xlabel('Time [ms]')
    ax[idx].set_ylabel('Conductance [$\mu$S]')
    ax[idx].legend()

    print('Fit: {} mmHg, with adjusted R2={}'.format(value, adjusted_coeff_determ(g_data_on[value].y, y_fit[idx], len(g_data_on[value].x), 8)))

In [ ]:
#Calculate the state variables
def evaluate_symfit(func_list):
    return [func.subs({globals()[key]: value for key, value in fit_result.params.items()}) for func in func_list]

alpham, betam, alphah, betah = [evaluate_symfit(x) for x in [alpham, betam, alphah, betah]]
m_inf, tau_m, h_inf, tau_h = [evaluate_symfit(x) for x in [m_inf, tau_m, h_inf, tau_h]]

In [ ]:
#Plot the state variables
fig, ax = plt.subplots(2,2,figsize= (15,10))
fig.suptitle('Fitted values of state variables')

ax[0][0].plot(P_arr, m_inf, 'o')
ax[0][0].set_ylabel('$m_{\infty}$')
ax[0][0].grid()
ax[0][0].set_title('Activation')

ax[1][0].plot(P_arr, tau_m, 'o')
ax[1][0].set_ylabel('$\\tau_m$ [ms]')
ax[1][0].set_xlabel('Pressure [mmHg]')
ax[1][0].grid()

ax[0][1].plot(P_arr, h_inf, 'o')
ax[0][1].set_ylabel('$h_{\infty}$')
ax[0][1].grid()
ax[0][1].set_title('Inactivation')

ax[1][1].plot(P_arr, tau_h, 'o')
ax[1][1].set_ylabel('$\\tau_h$ [ms]')
ax[1][1].set_xlabel('Pressure [mmHg]')
ax[1][1].grid()